# Getting started with Lightdash

The Lightdash Python client allows you query data from the Lightdash Semantic Layer directly from Python. For example, you can use it to query your governed metrics for:
- Creating data visualisations and interactive apps (matplotlib, plotly, etc)
- Exploring data in a notebook (hex, jupyter, etc)
- Pulling data into orchestration pipelines (dagster, airflow, etc)

## Installation

```bash
pip install lightdash
```

## 1. Setup Lightdash Client

Make sure you have the following environment variables set:
- `LIGHTDASH_ACCESS_TOKEN`
- `LIGHTDASH_PROJECT_UUID`
- `LIGHTDASH_INSTANCE_URL`

Then you can create a Lightdash client:

In [2]:
from lightdash import Client

import os
access_token = os.getenv("LIGHTDASH_ACCESS_TOKEN")
project_uuid = os.getenv("LIGHTDASH_PROJECT_UUID")
instance_url = os.getenv("LIGHTDASH_INSTANCE_URL")

client = Client(
    instance_url=instance_url,
    access_token=access_token,
    project_uuid=project_uuid
)

## 2. Explore the semantic layer

### 2a. Find a model

You can use tab completion on `client.models` to find relevant models. If you print out the model in a notebook you'll get a rich preview including the model description and available metrics + dimensions

In [3]:
orders = client.models.dbt_orders

In [4]:
orders

Model(dbt_orders: This table contains information on all the confirmed orders and their status)

### 2B. Find a relevant metric

Use tab completion with `.metrics` to find relevant metrics on the model.

In [5]:
orders.metrics

In [6]:
orders.metrics.sum_of_basket_total

Metric(sum_of_basket_total: Total sales revenue from all orders.)

## 3. Run a query

### 3a. Run a simple query

Use the SDK query builder to write a semantic layer query. Use tab to auto-complete metric/dimension names

In [7]:
# Build the query
query = (orders.query()
     .metrics(orders.metrics.sum_of_basket_total)
     .dimensions(orders.dimensions.browser))

# Execute the query and print as a dataframe. You can also use .to_records .to_json .to_json_string
query.to_df()

,Browser,Revenue
0,safari,3660279
1,chrome,6791811
2,edge,2287038
3,firefox,459995
4,internet_explorer,238994
5,lynx,48820
6,netscape,92088


### 3b. Sorting

In [8]:
# Build the query
query = (orders.query()
     .metrics(orders.metrics.sum_of_basket_total)
     .dimensions(orders.dimensions.browser)
     .sort(orders.metrics.sum_of_basket_total.desc()))

# Execute the query and print as a dataframe. You can also use .to_records .to_json .to_json_string
query.to_df()

,Browser,Revenue
0,chrome,6791811
1,safari,3660279
2,edge,2287038
3,firefox,459995
4,internet_explorer,238994
5,netscape,92088
6,lynx,48820


### 3c. Filtering

In [9]:
f = (orders.dimensions.partner_name == 'Redwood Ranch')

query = (orders.query()
     .metrics(orders.metrics.sum_of_basket_total)
     .dimensions(orders.dimensions.browser)
     .dimensions(orders.dimensions.partner_name)
     .filter(f))

# Execute the query and print as a dataframe. You can also use .to_records .to_json .to_json_string
query.to_df()

,Browser,Partner name,Revenue
0,chrome,Redwood Ranch,1595232
1,safari,Redwood Ranch,873290
2,edge,Redwood Ranch,554889
3,internet_explorer,Redwood Ranch,55904
4,netscape,Redwood Ranch,24285
5,lynx,Redwood Ranch,9886
6,firefox,Redwood Ranch,105184


## 4. Run Raw SQL

In [16]:
client.sql('SELECT 1 as hello').to_df()

,hello
0,1
